In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [8]:
data=pd.read_csv('Churn_Modelling.csv')
##preprocess 
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
#encode categorical variables
label_encode_gender=LabelEncoder()
data["Gender"]=label_encode_gender.fit_transform(data['Gender'])

#one hot encode geographic columns
from sklearn.preprocessing import OneHotEncoder
one_hot_encode_geo=OneHotEncoder()
geo_encoder=one_hot_encode_geo.fit_transform(data[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encode_geo.get_feature_names_out(["Geography"]))
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


x=data.drop('Exited',axis=1)
y=data['Exited']


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
scaler=StandardScaler() 
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file)

with open('one_hot_encode_geo.pkl','wb') as file:
    pickle.dump(one_hot_encode_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(
        neurons,
        activation="relu",
        input_shape=(x_train.shape[1],)
    ))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


model = KerasClassifier(
    model=create_model,
    epochs=50,
    batch_size=10,
    verbose=0
)

param_grid = {
    "model__neurons": [16, 32, 64],
    "model__layers": [1, 2],
    "epochs": [50, 100]
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    n_jobs=1
)

grid_result = grid.fit(x_train, y_train)

print(
    "Best: %f using %s"
    % (grid_result.best_score_, grid_result.best_params_)
)


d:\ML\churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\ML\churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\ML\churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\ML\churn\venv\Li

Best: 0.854999 using {'epochs': 100, 'model__layers': 1, 'model__neurons': 16}
